### Generalized Least Squares Practice Assignment

In this assignment, we will walkthrough using genearlized least squares (GLS) to analyze data and why it is useful method in analyzing data. 

For this assignment we will use global warming data from the r package `faraway`. This data set contains the Average Northen Hemisphere Temperature from 1856-2000 and eight climate proxies from 1000-2000AD. 

To begin, we will analyze the data using ordinary least squares (OLS). OLS assumes errors in the regression model have the same variance and are uncorrelated. Let's see if this is a valid assumption for our data!  

First, let's load our data set.

In [1]:
load('globwarm.rda')
globwarm <- na.omit(globwarm)

Then, we will use `lm()` to fit a linear model. 

In [2]:
lmod <- lm(nhtemp ~ wusa + jasper + westgreen + chesapeake + tornetrask + urals + mongolia + tasman, data = globwarm)

We can use `residuals()` to measure the error between the predicted value of our model and the observed value, and use `cor()` to see if our errors are correlated.

In [3]:
# find the number of observations
n <- length(residuals(lmod)) 

# measures between the first and last observations
cor(residuals(lmod)[-1], residuals(lmod)[-n])

[1] 0.583339

We see that the errors are correlated. A proxy to see how the errors behave is to look at the residuals. Looking at the correlation between the residuals lagged by one year. Years 2 to n and years 1 to (-n). Taking the correlation between the year and the subsequent years residuals.

Therefore fitting normal linear model will violate the correlation errors assumption. We will now use GLS to model the data.

Recall from the lecture on GLS that we need to estimate big Sigma. This will tell us the structure we are expecting from the correlation between these errors. We will then use that to be able to fit a model that incorporates that correlation appropriately. 

Assuming the errors take a simple auto-regressive form such that each error is correlated with the prior:

$$ \epsilon_{i+1} = \phi\epsilon_i + \delta_i$$ such that, $$\delta_i \sim N(0, \tau^2)$$ 

$\phi$ is going to be the correlation coefficient and we have it added to an additional error term $\delta_i$ we expect since it is normally distributed. 

We estimated $\phi$ above using `cor(residuals(lmod)[-1], residuals(lmod)[-n])`, which is the correlation we are estimating between the i^th error and the i^th+1 error an we got the value of 0.583339

Under the assumption $$\Sigma_{ij} =  \phi^{|i-j|}$$ we can estimate it like this:

In [4]:
X <- model.matrix(lmod)  # design matrix 

Sigma <- diag(n) # I want a matrix that is nxn. 0 on off diagonals and 1 on the diagonals. 
Sigma <- 0.5833^abs(row(Sigma)- col(Sigma))

In [5]:
y <- globwarm$nhtemp

Sigma_inv <- solve(Sigma)
XTX_inv <- solve(t(X)%*% Sigma_inv %*% X)
betahat <- XTX_inv %*% t(X) %*% Sigma_inv %*% y # class derivation 
betahat

(Intercept),-0.234134783
wusa,0.068425906
jasper,-0.218438446
westgreen,0.003880871
chesapeake,-0.014952072
tornetrask,0.057691347
urals,0.222078555
mongolia,0.055247801
tasman,0.122999856


What is the correlation now?

In [6]:
res  <- y - X%*% betahat
cor(res[-1], res[-n])

[1] 0.5887776

Another method is to use the P matrix we derived during the lecture: 

In [7]:
P <- chol(Sigma)  # square root of sigma matrix / choleski decomposition 
P_inv <- solve(t(P))

PX <- P_inv %*% X
Py <- P_inv %*% y 

lm(Py ~ PX -1) # -1 is because PX has an intercept in it and I don't want it to fix in that additional intercept. He we regressed the Prime values. 


Call:
lm(formula = Py ~ PX - 1)

Coefficients:
PX(Intercept)         PXwusa       PXjasper    PXwestgreen   PXchesapeake  
    -0.234135       0.068426      -0.218438       0.003881      -0.014952  
 PXtornetrask        PXurals     PXmongolia       PXtasman  
     0.057691       0.222079       0.055248       0.123000  


Finally, the `nlme` package in R has a function to perform GLS: 

In [8]:
library(nlme)

glmod <- gls(nhtemp ~ wusa + jasper + westgreen + chesapeake + tornetrask + urals + mongolia + tasman, correlation = corAR1(form = ~ year), data = globwarm)

summary(glmod)

Generalized least squares fit by REML
  Model: nhtemp ~ wusa + jasper + westgreen + chesapeake + tornetrask +      urals + mongolia + tasman 
  Data: globwarm 
        AIC       BIC   logLik
  -108.2074 -76.16822 65.10371

Correlation Structure: AR(1)
 Formula: ~year 
 Parameter estimate(s):
      Phi 
0.7109922 

Coefficients:
                  Value  Std.Error   t-value p-value
(Intercept) -0.23010624 0.06702406 -3.433188  0.0008
wusa         0.06673819 0.09877211  0.675678  0.5004
jasper      -0.20244335 0.18802773 -1.076668  0.2835
westgreen   -0.00440299 0.08985321 -0.049002  0.9610
chesapeake  -0.00735289 0.07349791 -0.100042  0.9205
tornetrask   0.03835169 0.09482515  0.404446  0.6865
urals        0.24142199 0.22871028  1.055580  0.2930
mongolia     0.05694978 0.10489786  0.542907  0.5881
tasman       0.12034918 0.07456983  1.613913  0.1089

 Correlation: 
           (Intr) wusa   jasper wstgrn chespk trntrs urals  mongol
wusa       -0.517                                        

We see `Phi` is estimated to be 0.7109922. 

This is for a one year lag which is a pretty high correlation but make sense in terms of year to year temperature. 


We can now calculate the confidence interval on that using the `intervals` function.

In [9]:
intervals(glmod, which = "var-cov") # conf-int for the phi value

Approximate 95% confidence intervals

 Correlation structure:
        lower      est.     upper
Phi 0.5099658 0.7109922 0.8383787
attr(,"label")
[1] "Correlation structure:"

 Residual standard error:
    lower      est.     upper 
0.1540697 0.2045720 0.2716284 

Safe to say to that there is definitely some correlation that is occurring between these errors so we did the right thing to fit a GLS model for this.